### The following are queries for a SQLite database, which can be run in either SQLite or Python

This is a generic Python function to query the SQLite database and display select data in a pandas dataframe.

In [25]:
#Import the SQLite3 library and name it 'sql'
import sqlite3 as sql
#Import the Pandas library and name it 'pd'
import pandas as pd

#Assign an SQL query to meaningful variable
#Calculate file sizes using query explained above
#'query_name' = 'SQL query'
calculate_file_sizes = "SELECT COUNT(ID) AS tif_count, MIN(SIZE) AS min_tif_size_B, MAX(SIZE) AS max_tif_size_B, MAX(SIZE)/1024/1024 AS max_tif_size_MB, AVG(SIZE)/1024/1024 AS avg_tiff_size_MB, AVG(SIZE)/1024/1024/2012/1024 AS avg_tiff_size_TB, SUM(SIZE)/1024/1024/1024/1024 AS total_tiff_size_TB FROM source_AAC_Siegfried_201908 WHERE EXT = 'tif'"
#Display the analysis_status and analysis_folder columns from the analysis_log table for all folders marked 'Complete'
folders_analysed = "SELECT analysis_status, analysis_folder FROM analysis_log WHERE analysis_status = 'Complete'"
#Display the EXTENSION_MISMATCH and FILE_PATH columns from the analysis_log table for all files marked 'True'
ext_mismatches = "SELECT EXTENSION_MISMATCH, FILE_PATH FROM source_AAC_Siegfried_201908 WHERE EXTENSION_MISMATCH = 'TRUE'" 

def run_query(query):
    #Create the connection to the database (saved as PIMMS.db)
    con = sql.connect(r'../DatabaseDesign/PIMMS.db')
    #Create the dataframe from a SQL query defined above
    df = pd.read_sql_query(query, con)
    #Print the results of the query
    print(df)
    #Close the connection to the database
    con.close()

#Run the Python function using the SQL query in the query variable assigned above
#'query_function'('query_name')
run_query(calculate_file_sizes)
run_query(folders_analysed)
run_query(ext_mismatches)

   analysis_status                                    analysis_folder
0         Complete                            AAS Storage\Korean (HT)
1         Complete                      AAS Storage\AAS Legacy Images
2         Complete            AAS Storage\AAS Reading Room catalogues
3         Complete                             AAS Storage\Adi Granth
4         Complete                       AAS Storage\Arabic (DL - CB)
5         Complete                          AAS Storage\Avestan (USW)
6         Complete              AAS Storage\Awaiting DIPS Publication
7         Complete                      AAS Storage\Chinese (SC - EG)
8         Complete           Greek MSS Digitisation\Greek MSS Phase 3
9         Complete                                             SEAsia
10        Complete                       AAS Storage\West Africa (MW)
11        Complete                     AAS Storage\Japanese (HT - YO)
12        Complete                            AAS Storage\Coptic (IT)
13        Complete  

This is a query to run calculations of file sizes for storage planning and analysis, to be run directly in SQLite. References: https://www.w3schools.com/sql/sql_count_avg_sum.asp and https://www.techspot.com/news/68482-quickly-convert-between-storage-size-units-kb-mb.html

In [ ]:
SELECT 
	COUNT(ID) AS tif_count,
	MIN(SIZE) AS min_tif_size_B,
	MAX(SIZE) AS max_tif_size_B,
	MAX(SIZE)/1024/1024 AS max_tif_size_MB,
	AVG(SIZE)/1024/1024 AS avg_tiff_size_MB,
	AVG(SIZE)/1024/1024/2012/1024 AS avg_tiff_size_TB,
	SUM(SIZE)/1024/1024/1024/1024 AS total_tiff_size_TB
FROM source_AAC_Siegfried_201908
WHERE EXT = 'tif';

### The following are SQL queries to join multiple tables based on matching or related elements. They can be run directly in DB Browser for SQLite and exported as csv or saved as 'views'.

This is a SQLite query to join columns from two tables in the database based on common fields. Reference: https://www.sqlitetutorial.net/sqlite-inner-join/. 

In [ ]:
SELECT 'source_AMEMM_SelfAnalysis'.ms_shelfmark, 'source_AMEMM_SelfAnalysis'.pro_title, 'source_AMEMM_SelfAnalysis'.ms_location, 'source_AMEMM_SelfAnalysis'.pro_details,  source_AMEMM_IAMS.*
FROM 'source_AMEMM_SelfAnalysis'
INNER JOIN source_AMEMM_IAMS ON 'source_AMEMM_SelfAnalysis'.ms_shelfmark = source_AMEMM_IAMS.'Record reference'
WHERE source_AMEMM_IAMS.Languages = 'French';

This is a SQL query to join columns from three source tables in the database based on common fields. Reference: https://stackoverflow.com/questions/3492904/mysql-select-all-columns-from-one-table-and-some-from-another-table. 

In [ ]:
SELECT 'ingest_Royal I Finished combined'.*, source_AMEMM_HMDAnalysis.*
FROM 'ingest_Royal I Finished combined'
INNER JOIN source_AMEMM_HMDAnalysis ON 'ingest_Royal I Finished combined'.shelfmark = source_AMEMM_HMDAnalysis.shelfmark;

This is a SQLite query to join columns from three target tables in the database, which have had their data normalised.

In [ ]:
SELECT folder_history.folder_path AS 'folderpath', items_aleph.*
FROM items_aleph
INNER JOIN folder_history ON folder_history.folder_id = folders_items.folder_id
INNER JOIN folders_items ON folders_items.system_num = items_aleph.system_num;

### The following are SQL and Python scripts to copy select data from source tables into target tables

This is a SQLite query to select data from the source tables and copy it into correct columns of the associated target tables. Reference: https://www.w3schools.com/sql/sql_insert_into_select.asp

In [ ]:
#Insert data into each of the columns of the target table (twitter_analytics)
INSERT INTO twitter_analytics (permalink, tweet_text, impressions, engagment_rate, replies, retweets, likes, link_clicks)
#Select the data from each of the columns from the corresponding source table (source_Twitter_Analytics)
SELECT "Tweet permalink", "Tweet text", "impressions", "engagement rate", "replies", "retweets", "likes", "url clicks" 
FROM source_Twitter_Analytics 
#Add a constraint to the selected data (the number of URL clicks is higher than 50)
WHERE "url clicks" > 50;